Differentiaalvergelijken voor verkeersdoorstroming

Utrecht CS, Oorsprongpark en Transferium USP op lijn 28 in Utrecht. Traject A is dus UCS->Oorspr, B is Oospr->USP

Voor dit traject gaan we uit van (op traject A) 400 auto/uur respectievelijk (op traject B) 650 auto/uur, met beide 6 bussen/uur. De vaste instroom is 90 respectievelijk 20 auto's / uur, met een uitstroom van 50% op beide trajecten. De maximum capaciteit voor A is 600 a/u, en voor B 1400 a/u, waarbij de instroom 5% van de beschikbare capaciteit is. Tot slot is de impact van het matrix-bord dat 10% van het aantal auto's per uur op traject B van het aantal auto's op traject A zal worden afgetrokken.

Differential vergelijking:  
c = beschikbare capaciteit  
b = instroming van traject b  
Traject A: dW(t)/dt = W(t) * 0.5 + 90 + c(t) * 0.05 - b(t) * 0.1  
Traject B: dW(t)/dt = W(t) * 0.5 + 20 + c(t) * 0.05  

Euler:  
Traject A: W(t + h) = W(t) * 0.5 + 90 + c(t) * 0.05 - b(t) * 0.1  
Traject B: W(t + h) = W(t) * 0.5 + 20 + c(t) * 0.05  

In [1]:
import matplotlib.pyplot as plt
import ipywidgets as widgets

In [2]:
class Euler():
    def __init__(self, stapgrootte, begin_traject_a, begin_traject_b, max_capaciteit_traject_a, max_capaciteit_traject_b,
                a_vaste_instroming, b_vaste_instroming):
        self.stapgrootte = stapgrootte
        self.autos_per_uur = [(begin_traject_a, begin_traject_b)]
        self.max_capaciteit_traject_a = max_capaciteit_traject_a
        self.max_capaciteit_traject_b = max_capaciteit_traject_b
        self.a_vaste_instroming = a_vaste_instroming
        self.b_vaste_instroming = b_vaste_instroming
        
    def euler_stap(self):
        # Bepaal met euler wat er uit traject b hoort te komen
        euler_traject_b = self.traject_b(self.autos_per_uur[-1][1])
        
        # Traject b mag niet over zijn maximum capaciteit heen
        if euler_traject_b > self.max_capaciteit_traject_b:
            euler_traject_b = self.max_capaciteit_traject_b
        
        # 10% van traject b wordt afgetrokken van traject a en bepaal met euler wat er uit traject a hoort te komen
        euler_traject_a = self.traject_a(self.autos_per_uur[-1][0], euler_traject_b)
        
        # Traject a mag niet over zijn maximum capaciteit heen
        if euler_traject_a > self.max_capaciteit_traject_a:
            euler_traject_a = self.max_capaciteit_traject_a
        
        # Voeg de uitkomst bij een lijst.
        self.autos_per_uur.append((euler_traject_a, euler_traject_b))
        
    def traject_a(self, start, b_instroming):
        vaste_instroom = self.a_vaste_instroming / self.stapgrootte  # traject a heeft een vaste instroom van 90 auto/uur.
        # 5% van de beschikbare capaciteit stroomt in.
        instroom = ((self.max_capaciteit_traject_a - start) * 0.05) / self.stapgrootte
        uitstroom = start - start * (1 - 0.5 / self.stapgrootte)  # 50% gaat weg
        b_instroming *= 0.1 / self.stapgrootte # 10% van traject b wordt afgetrokken van traject a

        # We voegen 90 en 5% beschikbare capaciteit toe.
        # Daarna halen we 10% van b_instroming weg en dan halen we hier weer 50% van af.
        return start - uitstroom + vaste_instroom + instroom - b_instroming

    def traject_b(self, start):
        vaste_instroom = self.b_vaste_instroming / self.stapgrootte  # traject b heeft een vaste instroom van 20 auto/uur.
        # 5% van de beschikbare capaciteit stroomt in.
        instroom = ((self.max_capaciteit_traject_b - start) * 0.05) / self.stapgrootte  
        uitstroom = start - start * (1 - 0.5 / self.stapgrootte)  # 50% gaat weg

        # Eerst halen we 50% af van de vorige waarde
        # We voegen 20 en 5% beschikbare capaciteit toe.
        return start - uitstroom + vaste_instroom + instroom
    
    def plot(self):
        plt.plot(self.autos_per_uur)
        plt.title('Verkeersdoorstroming')
        plt.ylabel('auto/uur')
        plt.xlabel('Stappen')
        plt.legend(['Traject A', 'Traject B'])
    
def plot_euler(stappen, stapgrootte, start_a, start_b, max_cap_a, max_cap_b, a_vaste_in, b_vaste_in):
    euler = Euler(stapgrootte, start_a, start_b, max_cap_a, max_cap_b, a_vaste_in, b_vaste_in)
    for i in range(stappen*stapgrootte):
        euler.euler_stap()
    euler.plot()

In [3]:
class Heun(Euler):
    def euler_stap(self):
        # bepaal met euler wat er uit traject b hoort te komen
        euler_traject_b = self.traject_b(self.autos_per_uur[-1][1])
        
        # Traject b mag niet over zijn maximum capaciteit heen
        if euler_traject_b > self.max_capaciteit_traject_b:
            euler_traject_b = self.max_capaciteit_traject_b
        
        # 10% van traject b wordt afgetrokken van traject a en bepaal met euler wat er uit traject a hoort te komen
        euler_traject_a = self.traject_a(self.autos_per_uur[-1][0], euler_traject_b)
        
        # Traject a mag niet over zijn maximum capaciteit heen
        if euler_traject_a > self.max_capaciteit_traject_a:
            euler_traject_a = self.max_capaciteit_traject_a
        
        # Voeg de uitkomst bij een lijst.
        return (self.autos_per_uur[-1][0] - euler_traject_a,self.autos_per_uur[-1][1] -  euler_traject_b)
        
    def heun_stap(self):
        euler_a, euler_b = self.euler_stap()
        # Bereken hier heun
        heun_b = self.heun_traject_b(self.autos_per_uur[-1][1], euler_b)
        heun_a = self.heun_traject_a(self.autos_per_uur[-1][0], euler_a, heun_b)
        
        # Voeg de uitkomst bij een lijst.
        self.autos_per_uur.append((heun_a, heun_b))
    
    def heun_traject_b(self, start, euler_b):
        vaste_instroom = self.b_vaste_instroming  # traject b heeft een vaste instroom van 20 auto/uur.
        # 5% van de beschikbare capaciteit stroomt in.
        instroom = (self.max_capaciteit_traject_b - start) * 0.05
        uitstroom = start - start * 0.5  # 50% gaat weg
        
        return start + (self.stapgrootte / (start - uitstroom + vaste_instroom + instroom) - euler_b) / 2
        
    def heun_traject_a(self, start, euler_a, b_instroming):
        vaste_instroom = self.a_vaste_instroming  # traject a heeft een vaste instroom van 90 auto/uur.
        # 5% van de beschikbare capaciteit stroomt in.
        instroom = (self.max_capaciteit_traject_a - start) * 0.05
        uitstroom = start - start * 0.5  # 50% gaat weg
        b_instroming *= 0.1 # 10% van traject b wordt afgetrokken van traject a

        return start + (self.stapgrootte / (start - uitstroom + vaste_instroom + instroom - b_instroming) - euler_a) / 2
    
def plot_heun(stappen, stapgrootte, start_a, start_b, max_cap_a, max_cap_b, a_vaste_in, b_vaste_in):
    heun = Heun(stapgrootte, start_a, start_b, max_cap_a, max_cap_b, a_vaste_in, b_vaste_in)
    for i in range(stappen*stapgrootte):
        heun.heun_stap()
    heun.plot()

In [4]:
plot_type = plot_heun # Plot type kan plot_heun of plot_euler zijn. Plot heun werkt helaas niet.

widgets.interact(plot_type, 
                 stappen=widgets.IntSlider(min=1, max=180, value=10, continuous_update=False), 
                 stapgrootte=widgets.IntSlider(min=1, max=200, step=1, value=100, continuous_update=False),
                 start_a=widgets.IntSlider(min=1, max=1000, value=400, continuous_update=False),
                 start_b=widgets.IntSlider(min=1, max=2000, value=650, continuous_update=False),
                 a_vaste_in=widgets.IntSlider(min=1, max=1000, value=90, continuous_update=False),
                 b_vaste_in=widgets.IntSlider(min=1, max=1000, value=20, continuous_update=False),
                 max_cap_a=widgets.IntSlider(min=300, max=1000, value=600, continuous_update=False),
                 max_cap_b=widgets.IntSlider(min=550, max=2000, value=1400, continuous_update=False))
"""
Stappen: Aantal stappen dat wordt gesimuleerd.  
Stapgrootte: Hoe groot een stap is.  
Start A: Het aantal auto/uur traject a mee begint.  
Start B: Het aantal auto/uur traject b mee begint.  
Max Cap A: Maximum capaciteit van traject a  
Max Cap B: Maximum capaciteit van traject b  
A Vaste In: Vaste instroming van traject a  
B Vaste In: Vaste instroming van traject b  
"""

interactive(children=(IntSlider(value=10, continuous_update=False, description='stappen', max=180, min=1), Int…

'\nStappen: Aantal stappen dat wordt gesimuleerd.  \nStapgrootte: Hoe groot een stap is.  \nStart A: Het aantal auto/uur traject a mee begint.  \nStart B: Het aantal auto/uur traject b mee begint.  \nMax Cap A: Maximum capaciteit van traject a  \nMax Cap B: Maximum capaciteit van traject b  \nA Vaste In: Vaste instroming van traject a  \nB Vaste In: Vaste instroming van traject b  \n'

Conclusie:

Aan de hand van de grafiek kunnen we zien dat de verhouding langzaam daalt naar een balans. Hierdoor kunnen we zien dat na een tijdje het aantal auto/uur stabiel op ongeveer 190 op traject b blijft en ongeveer 160 op traject a.  

Als we dus aannemen dat het aantal bussen constant is, dan is de verhouding na een tijdje: 6 bussen op 190 auto/uur op traject a en 6 bussen op 120 auto/uur op traject b.